In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Geometric Properties of Surfaces

The use of nearest neighbor approximation allowed us to present the geometric properties that captured the shape of a curve, quantities were derived by enacting small changes to the evaluation parameter (`t+delta_t`), and observing the variations of points on the curve that result.

In this section, we define a number of analogous properties that describe surface shape. In examining these properties, we rely upon the same basic set of techniques used for curves.

Two sets of properties will be presented: 
* those related to the local tangent vector
* those that quantify curvature. 

As we saw with the Frenet frame of a curve, many of the important features of a surface are succinctly captured by an orthonormal frame associated with a surface point. For a surface this frame is called ***the Darboux frame***.

As with curves, we will find a specific set of ***tangent vectors***, which can then be used to define ***a whole plane of tangent vectors*** with a ***normal vector***. Tangent vectors are also used in computing the ***surface area***.

Surface ***curvature***, on the other hand, is defined by taking variations of position along the surface itself. This process results in a quantification of the amount of local “bending” at work at a given location on and in a given direction along the surface. Here we will find an analog to the osculating circle we defined for curves.

## Tangent Vectors

Just as a parameterized curve manifests a single tangent vector for any parameter `t`, there are ***two tangent vectors related to a parameterized surface at a point described by a u,v pair***. Each captures the variations of the positions on the surface as one of the parameters is varied while the other remains fixed.

The variations described here are happening by moving either vertically or horizontally in parameter space, directions which follow the surface isocurves, and do not correspond to any fixed direction in world space.

In code, the Decod.es Surface class defines a private method for calculating tangent vectors:

In [ ]:
"""
Tangent Vectors by Nearest Neighbors
Returns the evaluated Point and Vecs oriented toward the nearest neighbors along the u and v 
directions of a Surface.
"""
def nearest_neighbors(srf,u,v,include_negs = False):
    pt = Point(srf.func(u,v))
    
    du, dv = srf.tol_u/100.0, srf.tol_v/100.0
    vec_u = Vec( pt, srf.func( u+du, v ) )
    vec_v = Vec( pt, srf.func( u, v+dv ) )
    return pt, vec_u, vec_v

If two vectors are not parallel, then they span an entire plane. Not surprisingly, the plane spanned by two tangent vectors defined above is called the ***tangent plane***. If a tangent plane exists at every point on the surface, then the surface is called ***regular***.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P18.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Tangent Plane
Evaluates a Surface and returns the tangent Plane associated with the location determined by the 
given u and v parameters.
"""
def tangent_plane(srf,u,v):
    pt, vec_u, vec_v = srf.nearest_neighbors(u,v)
    return Plane(pt, vec_u.cross(vec_v) )

The dot products of the tangent vectors may be calculated, which determines a vector normal to the surface at any regular surface point, called the ***unit normal vector***.

Starting off with two tangent vectors dependent on the parameterization of the surface, we can assert that at every regular surface point, ***an entire plane of tangent vectors exists***. 

Amongst all the directions in this plane, there are two in particular that we should note, since they encapsulate useful geometric information about the local shape of the surface. These are called the ***principal curvature directions***. More on this in a bit.

### Surface Area

Just as the key to deriving the arc length of a curve was to express it as the sum of the lengths of many smaller line segments, so the first step in deriving the total area of a surface is to express it as the sum of smaller areas.

Taking these smaller areas as those that lie between consecutive isocurves, the approximate area of a surface patch is the area of the parallelogram between the tangent vectors.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P19.jpg" style="width: 600px; display: inline;">

### Normal Vector

The cross product of two tangent vectors determines a vector normal to the surface at any regular surface point, This gives the definition of the ***unit normal vector***.

The variation of the normal vector as it moves along the surface represents
one component of a moving frame analogous to the Frenet
frame on a curve.

A note: there are two faces to a tangent plane. For many surfaces, such as a sphere or cylinder, we can distinguish between these two according to their “outward-facing” or “inward-facing” direction relative to the surface. This allows us to bring up the important and somewhat enigmatic topological property of ***surface orientability***. Roughly, a surface is orientable if there is a parameterization such that its normals are either all outward-facing or all inward-facing.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P20.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P21.jpg" style="width: 200px; display: inline;">

There is an alternative method for caclulating the normal vector of a surface called the ***mesh normal***.

In Decod.es, given a Mesh `ngbrs` that describes the triangular faces that surround a patch of a Surface that centers on a Point, the discrete surface normal is calculated.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P22.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P23.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Discrete Surface Unit Normal
Given a Mesh ngbrs that describes the triangular faces that surround a Surface patch, this routine 
calculates the weighted sum of the face normals of the mesh in order to approximate the surface unit 
normal.
"""
normal = Vec(0,0,0)
for face_idx in range(len(ngbrs.faces)):
    pts = ngbrs.face_pts(face_idx)
    face_area = 0.5*Vec(pts[0],pts[1]).cross(Vec(pts[0],pts[2])).length 
    normal = normal + ngbrs.face_normal(face_idx)*face_area

normal = normal.normalized()

## Curvature

Just as curvature was a key quantity in measuring the shape of a curve, surface curvature contributes to a similar measurement for a surface.

In the case of a curve, curvature is described using a positive quantity that accounts for how much a curve deviates from a straight line. Here we used the osculating circle.

There are more quantities required to describe the curvature of a surface, and
***the signs of these quantities*** can come into play in different combinations
to describe a broader range of conditions.

We illuminate surface curvature quantities using the ***osculating paraboloid***, which is for surfaces what the osculating circle is for curves.

### The Osculating Paraboloid

Any smooth surface can be locally approximated by a paraboloid. 

For any surface point, there is an osculating paraboloid which glances the surface at this point and also shares the tangent plane and curvature behavior found there.

The standard classification of paraboloids is used as the classification of surface points.

Paraboloids are distinguished by the quantity $d = B^2 - 4AC$, with $d < 0$
being an ***elliptic paraboloid***, $d > 0$ a ***hyperbolic paraboloid***, and $d = 0$ indicating a ***parabolic cylinder***. 

On a given surface, ***a point is classified as hyperbolic, elliptic or parabolic*** according to which type describes the osculating paraboloid approximating the surface in a local neighborhood of that point. 

Colloquially, these classifications respectively describe ***double-curvature in the same direction***, ***double-curvature in opposite directions***, and ***single-curvature***.


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P30.jpg" style="width: 600px; display: inline;">

### The Normal Curvatures

Here we turn to the first of the curvature quantities: ***the normal curvatures***. 

Recall that the isoparametric curves of a surface represent a special case of
a curve on a surface, and that for any given position, we may find any number of curves that pass through it.

Take, for example, a plane that contains both a surface point and the unit normal vector of the surface at that point, called a ***normal plane***. Imagine that we intersect a normal plane with the surface on which it rests. The curve that results is called a ***normal curve***, and the curvature of this curve at the surface point is called a ***normal curvature***.

Look! 

***There are many possible normal curvatures for any given point on a surface***, which we can see by rotating a plane of intersection about the axis of the normal vector. Importantly, there is usually an orientation that registers the most amount of curvature, and one that registers the least. These ***maximum and minimum curvature values*** are called the ***principal curvatures***, and the directions of the normal planes that take these extreme values are called the ***principal curvature directions***.

Amazingly, it turns out that these directions are ***always perpendicular to
one another***.



<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P24.jpg" style="width: 200px; display: inline;">

The perpendicularity of the principal curvature directions is important in that the discovery of these two vectors allow us to determine ***the Darboux frame***: an orthonormal frame associated with a surface point. 

It is not, however, oriented with respect to the isoparametric lines of the surface, but rather ***oriented to the principal curvature lines tangent to the principal curvature directions***.

In Decod.es, we may produce this frame via the method `srf.eval_crv(u,v)`, which returns a CS that corresponds to the Darboux frame at the given position. This CS will find its origin on the evaluated Point, its x- and y-axis oriented in the principal curvature directions, and its z-axis coincident with the normal vector.



<table style="width:600px">
    <tr>
        <th colspan="2" style="text-align:left">*Basic Surface Properties*</th>
    </tr>
    <tr>
        <td style="width:20%">Elliptic</td>
        <td style="width:80%">An elliptic surface point has principal curvatures which have the same sign and are different from zero. This is sometimes called double-curvature in the same direction.</td>
    </tr>
    <tr>
        <td style="width:20%">Umbilic</td>
        <td style="width:80%">An umbilic surface point is a special case of an elliptic surface point, in which the principal curvatures are the same. In such cases, since the maximum and minimum coincide, all normal curvatures must be the same, and thus the surface bends the same amount in all directions about the surface point.</td>
    </tr>
    <tr>
        <td style="width:20%">Hyperbolic</td>
        <td style="width:80%">A hyperbolic surface point has principal curvatures with opposite signs. This can be called double-curvature in opposite directions.</td>
    </tr>
    <tr>
        <td style="width:20%">Parabolic</td>
        <td style="width:80%">If one principal curvature is zero and the other nonzero, we call the point parabolic, or say that it manifests single-curvature.</td>
    </tr>
    <tr>
        <td style="width:20%">Flat</td>
        <td style="width:80%">For a flat surface point, both principal curvatures are zero. Since it follows that all normal curvatures are zero, the osculating paraboloid degenerates to the tangent plane, and there is no bending.</td>
    </tr>
</table>

#### Elliptic
An elliptic surface point has principal curvatures which have the same sign and are different from zero. This is sometimes called double-curvature in the same direction.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P25.jpg" style="width: 200px; display: inline;">

#### Umbilic
An umbilic surface point is a special case of an elliptic surface point, in which the principal curvatures are the same. In such cases, since the maximum and minimum coincide, all normal curvatures must be the same, and thus the surface bends the same amount in all directions about the surface point.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P26.jpg" style="width: 200px; display: inline;">

#### Hyperbolic
A hyperbolic surface point has principal curvatures with opposite signs. This can be called double-curvature in opposite directions.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P27.jpg" style="width: 200px; display: inline;">

#### Parabolic
If one principal curvature is zero and the other nonzero, we call the point parabolic, or say that it manifests single-curvature.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P28.jpg" style="width: 200px; display: inline;">

#### Flat
For a flat surface point, both principal curvatures are zero. Since it follows that all normal curvatures are zero, the osculating paraboloid degenerates to the tangent plane, and there is no bending.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P29.jpg" style="width: 200px; display: inline;">

### Mean and Gaussian Curvature

The principal curvatures give rise to two other surface curvature quantities evaluated at a surface point. The mean curvature is the arithmetic mean of the principal curvatures and is denoted by $H$, while the Gaussian curvature is their product and is denoted by $K$.

By invoking an optional parameter `srf.eval_crv(u,v,True)`, we receive a Tuple containing the Darboux frame, along with quantities for the maximum principal curvature `k1`, minimum principal curvature `k2`, as well as the Gaussian curvature `K` and the mean curvature `H`. 

A common way of identifying special types of surfaces is by considering restrictions on the mean or Gaussian curvatures. 

For example, a surface that has mean curvature zero at every point is termed a ***minimal surface***. 

Another notable class of surfaces are those with ***constant Gaussian curvature***, such as a sphere which has constant positive Gaussian curvature (the smaller the sphere, the larger its curvature). Rotational surfaces are a rich source of surfaces with constant negative Gaussian curvature.

Developable surfaces have zero Gaussian curvature, with one of the principal directions at every surface point aligned with the straight line ruling through that point.



In [ ]:
"""
Gaussian Curvature Heat Map
"""
msh = surf.to_mesh(divs_u=res_u,divs_v=res_v)
u_val = Interval(surf.u0, surf.u1).divide(res_u,True)
v_val = Interval(surf.v0, surf.v1).divide(res_v,True)

gaussian_curv = [surf.deval_curv(u,v,True)[3] for v in v_val for u in u_val]

colorA, colorB, colorC = Color(.97,.97,.97), Color(1, 0, 0), Color(0, 0, 1)
min_curv, max_curv = min(gaussian_curv), max(gaussian_curv)
denom = max(abs(min(gaussian_curv)), abs(max(gaussian_curv)))

meshes = Mesh.explode(msh)
for i, mesh in enumerate(meshes):
    if gaussian_curv[i] > 0: 
        mesh.set_color(Color.interpolate(colorA,colorB,gaussian_curv[i]/denom))
    else: 
        mesh.set_color(Color.interpolate(colorA,colorC,-gaussian_curv[i]/denom))